In [1]:
import numpy as np
from scipy.stats import multivariate_normal

In [2]:
# from sklearn.mixture import GMM as GaussianMixture
from sklearn.mixture import GaussianMixture

In [3]:
from sklearn.cluster import KMeans

In [4]:
from random import randint

## Functions

## _GMM_

### LIB

In [5]:
def  find_clusters(items, features, n_cluster, n_iter):
    '''
        itemset: list indexes of images corresponding to video_img. Discard indexes where feature is None.
        usr_list: |U| users, each user is a sets of items.
        feature: gist or lab feature coressponding image in video_img.
    '''
    kmeans = KMeans(n_clusters = n_cluster, random_state=0).fit(features[items])
    label_init = kmeans.labels_

#     gmm = GaussianMixture(n_components=n_cluster, covariance_type = 'full', n_iter = n_iter, random_state=0)
    gmm = GaussianMixture(n_components=n_cluster, covariance_type = 'full', max_iter = n_iter, random_state=0)
    gmm.means_init = np.array([features[items[label_init == i]].mean(axis=0) for i in range(n_cluster)])
    gmm.fit(features[items])
    label_pred = gmm.predict(features[items])
    clusters = [np.where(label_pred == i)[0] for i in range(n_cluster)]

    means = gmm.means_
    covs = gmm.covariances_
#     covs = gmm.covars_
    weights = gmm.weights_
    
    p_x_y = np.vstack([np.apply_along_axis(multivariate_normal.pdf, 1, features[[items]], means[k], covs[k], True) \
                                                for k in range(n_cluster)]).T#     
    p_x = np.sum(p_x_y * np.array(weights).reshape(1, -1) , axis = 1).reshape(-1, 1) 
    print 'Log Likelihood: ', np.sum(np.log(p_x))
    return np.sum(np.log(p_x)), gmm

### My GMM

In [6]:
def cal_meancov(features, items, label, n_cluster):
    mean = [np.mean(features[items[label == i]], axis = 0) for i in range(n_cluster)]
    cov = [np.cov(features[items[label == i]], rowvar = 0) for i in range(n_cluster)]
    return mean, cov

In [7]:
def  my_gmm(items, features, n_cluster):
    kmeans = KMeans(n_clusters = n_cluster, random_state=0).fit(features[items])
    label_init = kmeans.labels_
    mean, cov = cal_meancov(features, items, label_init, n_cluster)
    p_y = np.array([1.0/n_cluster] * n_cluster).reshape(-1, 1)

    p_x_y = np.zeros((n_cluster, len(items))) # K * N
    p_y_x = np.zeros((n_cluster, len(items))) # K * N
    
    iter = 10
    converge = False
    while (iter > 0 and not converge):
        # e step
        for k in range(n_cluster):
            p_x_y[k, :] = np.apply_along_axis(multivariate_normal.pdf, 1, features[items], mean[k], cov[k], True)
        p_y_x = p_x_y * p_y # K * N
        p_x = np.sum(p_y_x, axis = 0).reshape(-1, 1) # N * 1 
        p_y_x = p_y_x / p_x.reshape(-1,) # K * N
        log_likelihood = np.sum(np.log(p_x))

        print 'Log Likelihood: ', log_likelihood
        
        # m step       
        sum_weight = np.sum(p_y_x, axis = 1).reshape(-1, 1)  # K x 1
        mean = np.dot(p_y_x, features[items]) * (1.0/sum_weight.reshape(-1,1)) # K x F
        x_submean = np.array([features[items] - mean[k] for k in range(n_cluster)])
        cov = [np.dot(np.multiply(x_submean[k].T, p_y_x[k]), x_submean[k]) / sum_weight[k] for k in range(n_cluster)] # K x F x F

        p_y = sum_weight / len(items)  # K x 1
        
        iter -= 1
    print 'Log likelihood: ', np.sum(np.log(p_x))
    return np.sum(np.log(p_x)), (p_y, mean, cov)

## Debug

In [8]:
# log_prob1, _ = my_gmm(np.array(range(N)), data, n_cluster = K)

In [9]:
# for n_iter in range(1, 10):
#     log_prob0, _ = find_clusters(np.array(range(N)), data, n_cluster = K, n_iter = n_iter)

## Test

In [10]:
''' GLOBAL VARIABLES '''
log_prob0s = []
log_prob1s = []
TEST_CASE = 10
N, K = None, None
data = []

In [11]:
def test(N, K, data):
    print N, K
    
    print 'LIB'
    log_prob0 = None
    for n_iter in range(1, 10):
        log_prob0, _ = find_clusters(np.array(range(N)), data, n_cluster = K, n_iter = n_iter)

    print '\nMy GMM'
    log_prob1, _ = my_gmm(np.array(range(N)), data, n_cluster = K)
    return log_prob0, log_prob1

In [12]:
for i in range(TEST_CASE):
    print '\n------------TEST %d----------------' % i
    K = randint(3, 11)
    N = randint(1, 10) * (20 - K)
    
    data = []
    locs = (np.random.randint(10, size=K) + 1) * 100
    szs = (np.random.randint(15, size=K) + 1) * 10
    f_sz = 17
    print locs
    for i in range(K):
        data = data + list(np.random.normal(loc=locs[i], scale=1.0, size=(szs[i], f_sz)))
    data = np.array(data)
    N = data.shape[0]
    
    log_prob0, log_prob1 = test(N, K/2, data)
    log_prob0s.append(log_prob0)
    log_prob1s.append(log_prob1)
print '\n----------------------------------'
print 'AVG %d TEST %.4f %.4f' % (TEST_CASE, np.mean(log_prob0s),  np.mean(log_prob1s))


------------TEST 0----------------
[700 900 800 600 900 500]
390 3
LIB


/home/anla/Apps/anaconda2/lib/python2.7/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


Log Likelihood:  -11585.2393509
Log Likelihood:  -11553.4697967
Log Likelihood:  -11528.8540729
Log Likelihood:  -11507.2001552
Log Likelihood:  -11463.4343416
Log Likelihood:  -11348.4195935
Log Likelihood:  -10855.5412196
Log Likelihood:  -10855.4449411
Log Likelihood:  -10855.4448226

My GMM
Log Likelihood:  -10749.4561246
Log Likelihood:  -10742.2739943
Log Likelihood:  -10740.8032671
Log Likelihood:  -10739.9182156
Log Likelihood:  -10739.241183
Log Likelihood:  -10738.7471856
Log Likelihood:  -10738.3913892
Log Likelihood:  -10737.8007542
Log Likelihood:  -10736.2978762
Log Likelihood:  -10733.764256
Log likelihood:  -10733.764256

------------TEST 1----------------
[200 900 600]
210 1
LIB
Log Likelihood:  -6409.78186004
Log Likelihood:  -6409.78186004
Log Likelihood:  -6409.78186004
Log Likelihood:  -6409.78186004
Log Likelihood:  -6409.78186004
Log Likelihood:  -6409.78186004
Log Likelihood:  -6409.78186004
Log Likelihood:  -6409.78186004
Log Likelihood:  -6409.78186004

My GMM

In [13]:
for i in range(TEST_CASE):
    print '\n------------TEST %d----------------' % i
    K = randint(3, 11)
    N = randint(1, 10) * (20 - K)
    
    data = []
    locs = (np.random.randint(10, size=K) + 1) * 100
    szs = (np.random.randint(15, size=K) + 1) * 10
    f_sz = 17
    print locs
    for i in range(K):
        data = data + list(np.random.normal(loc=locs[i], scale=1.0, size=(szs[i], f_sz)))
    data = np.array(data)
    N = data.shape[0]
    
    log_prob0, log_prob1 = test(N, K/3, data)
    log_prob0s.append(log_prob0)
    log_prob1s.append(log_prob1)
print '\n----------------------------------'
print 'AVG %d TEST %.4f %.4f' % (TEST_CASE, np.mean(log_prob0s),  np.mean(log_prob1s))


------------TEST 0----------------
[500 300 500 600 800 600 700 400 400 700 100]
980 3
LIB
Log Likelihood:  -29142.3155704
Log Likelihood:  -29134.816563
Log Likelihood:  -29130.2775203
Log Likelihood:  -29127.1613494
Log Likelihood:  -29124.7394483
Log Likelihood:  -29122.6443142
Log Likelihood:  -29120.6769014
Log Likelihood:  -29118.6419632
Log Likelihood:  -29116.643169

My GMM
Log Likelihood:  -29324.687164
Log Likelihood:  -29146.4006491
Log Likelihood:  -29138.4511397
Log Likelihood:  -29133.899799
Log Likelihood:  -29130.8408774
Log Likelihood:  -29128.6017666
Log Likelihood:  -29126.8065343
Log Likelihood:  -29125.2396774
Log Likelihood:  -29123.7825199
Log Likelihood:  -29122.3754987
Log likelihood:  -29122.3754987

------------TEST 1----------------
[100 600 300 100 200 400 300 800 500 400]
830 3
LIB
Log Likelihood:  -25290.3735446
Log Likelihood:  -25266.0322228
Log Likelihood:  -25227.171929
Log Likelihood:  -25127.5654296
Log Likelihood:  -24568.9760301
Log Likelihood:  